In [2]:
%load_ext autoreload
%autoreload 2

In [78]:
# Reinforcement Learning
from src.reinforcement_learning.env import HyperHeuristicEnv 

# Hyperheuristic
from src.HyperHeuristic import HyperHeuristic

from src.initial_solution.ConstructiveHeuristic import ConstructiveHeuristic, RandomConstructive, GreedyRandomizedConstructive, GreedyConstructive
from src.operators.DestroyOperators import RandomRemove, WorstRemove 
from src.operators.RepairOperators import RandomRepair, GreedyRepair
from src.local_search.LocalSearch import FirstImprovement

from src.accept.SimulatedAnnealing import SimulatedAnnealing
from src.accept.RecordToRecordTravel import RecordToRecordTravel

from src.stop.MaxRuntimeOrNoImprovement import MaxRuntimeOrNoImprovement

# Instances
from src.KnapsackInstance import KnapsackInstance
from utils.benchmark import BinaryKnapsackBenchmark
from utils.knapsackSortFunctions import sortIndexesByProfitWeightDensity

# Misc libraries
import pandas as pd
import numpy as np
import random
import time
import re
import os

import plotly.express as px

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# **Hyperheuristic Construction**

In [4]:
def on_best(cand, rand):
    # Write your code here
    return cand

In [5]:
stop = MaxRuntimeOrNoImprovement(max_runtime = 60, max_iterations = 1000)

recordToRecordTravel = RecordToRecordTravel(
    start_threshold = 0.5,
    end_threshold = 0.1,
    step = 0.01,
    method = 'linear',
    cmp_best = True
)

# simulatedAnnealingCriteria = SimulatedAnnealing(
#     start_temperature = 100,
#     end_temperature = 0.001,
#     step = 0.99,
#     method = 'exponential'
# )

randomRemove1 = RandomRemove(0.2)
randomRemove2 = RandomRemove(0.4)
worstRemove1 = WorstRemove(0.2)
worstRemove2 = WorstRemove(0.4)
randomRepair = RandomRepair()
greedyRepair = GreedyRepair()

firstImprovement = FirstImprovement()

randomConstruct = RandomConstructive()
greedyConstruct = GreedyConstructive(sortIndexesByProfitWeightDensity)
greedyRandomizedConstruct = GreedyRandomizedConstructive(0.3)

rewards = [5,3,1,0.5]

In [6]:
hyperHeuristic = HyperHeuristic(stop = stop, acceptance = recordToRecordTravel, rewards = rewards, on_best = on_best)

hyperHeuristic.add_refinement(randomRemove1, 'randomRemove0.2')
hyperHeuristic.add_refinement(randomRemove2, 'randomRemove0.4')
hyperHeuristic.add_refinement(worstRemove1, 'worstRemove0.2')
hyperHeuristic.add_refinement(worstRemove2, 'worstRemove0.4')
hyperHeuristic.add_refinement(randomRepair, 'randomRepair')
hyperHeuristic.add_refinement(greedyRepair, 'greedyRepair')

hyperHeuristic.add_refinement(firstImprovement, 'firstImprovement')

hyperHeuristic.add_refinement(randomConstruct, 'randomConstruct')
hyperHeuristic.add_refinement(greedyConstruct, 'greedyConstruct')
hyperHeuristic.add_refinement(greedyRandomizedConstruct, 'greedyRandomizedConstruct')

# **Benchmark**

In [7]:
benchmarkPath = "../data/benchmark/binaryKnapsack"
benchmark = BinaryKnapsackBenchmark("binary", benchmarkPath)
instancesList = benchmark.getInstancesList()
np.random.shuffle(instancesList)

# **Q-Learning**

In [8]:
def instanceGenerator(instancesNames):
    while True:
        instanceName = np.random.choice(instancesNames, 1, replace = True)[0]
        instance = benchmark.parseInstance(instanceName)
        yield (instanceName, instance)

In [79]:
dirFiles = os.listdir()
if "train.txt" in dirFiles and "test.txt" in dirFiles: 
    with open('./train.txt', 'r') as f:
        train = f.read().strip().split("\n")

    with open('./test.txt', 'r') as f:
        train = f.read().strip().split("\n")
        
else:
    train, test = train_test_split(instancesList, test_size = 0.2)

    with open('./train.txt', 'w') as f:
        f.write("\n".join(train))

    with open('./test.txt', 'w') as f:
        f.write("\n".join(test))

    trainInstancesGenerator = instanceGenerator(train)

True


In [11]:
env = HyperHeuristicEnv(hyperHeuristic, trainInstancesGenerator)

In [13]:
from IPython.display import clear_output

q_tables = []
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_tables.append((0, np.array(q_table, dtype = np.double)))

# Hyperparameters
alpha_start = 0.5
alpha_end = 0.05
gamma = 0.6
epsilon = 0.1
n_epochs = 15000

# For plotting metrics
all_epochs = []

for i in range(1, n_epochs + 1):
    state = env.reset()

    totalEpochIterations, totalEpochReward, = 1, 0
    done = False
    
    while not done:
        alpha = alpha_start + (i-1) * (alpha_end - alpha_start)/n_epochs
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        totalEpochReward += reward
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = next_state
        totalEpochIterations += 1
        
    runningTime = env.get_running_time()

    all_epochs.append([totalEpochIterations, totalEpochReward, runningTime])

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        # print(q_table)
        q_tables.append((i, np.array(q_table, dtype = np.double)))

print("Training finished.\n")

Episode: 15000
Training finished.



In [15]:
trainingSummary = pd.DataFrame(data = all_epochs, columns = ['iterations', 'reward', 'runnnigTime'])
trainingSummary.head()

,iterations,reward,runnnigTime
0,1236,832.5,2.015615
1,2,0.5,0.018470
2,2,5.0,0.014452
3,1002,961.5,2.961271
4,2,0.5,0.018626


## **Evaluation**

In [16]:
columns = ['instanceName', 'optimumFO', 'bestFO', 'optimumRuntime', 'runtime', 'relativeError']
values = []

for instanceName in test:
    print(instanceName)
    instance = benchmark.parseInstance(instanceName)

    state = env.reset(instance)

    done = False

    while not done:
        action = np.argmax(q_tables[-1][1][state])
        state, reward, done, info = env.step(action)

    optimum = benchmark.getOptimalFO(instanceName)
    objective = env._hyperHeuristic.best_solution.objective(isMinimizing = False)
    runtime = env.get_running_time()
    optimumRuntime = benchmark.getOptimalRunningTime(instanceName)

    values.append([
        instanceName.replace('/test.in',''),
        optimum,
        objective,
        optimumRuntime,
        runtime,
        (optimum - objective)/optimum,
    ])

n_600_c_1000000_g_2_f_0.3_eps_0_s_300/test.in
n_800_c_10000000000_g_14_f_0.3_eps_0.1_s_200/test.in
n_800_c_10000000000_g_2_f_0.2_eps_0.1_s_200/test.in
n_800_c_10000000000_g_6_f_0.1_eps_0.001_s_200/test.in
n_600_c_100000000_g_14_f_0.1_eps_1e-05_s_100/test.in
n_1200_c_10000000000_g_10_f_0.1_eps_0.1_s_200/test.in
n_1000_c_100000000_g_2_f_0.3_eps_0_s_200/test.in
n_600_c_100000000_g_6_f_0.2_eps_0.0001_s_200/test.in
n_400_c_100000000_g_2_f_0.1_eps_1e-05_s_200/test.in
n_400_c_10000000000_g_6_f_0.1_eps_0.0001_s_100/test.in
n_400_c_1000000_g_6_f_0.2_eps_1e-05_s_100/test.in
n_400_c_1000000_g_6_f_0.1_eps_1e-05_s_200/test.in
n_1000_c_1000000_g_2_f_0.3_eps_0.01_s_300/test.in
n_800_c_10000000000_g_14_f_0.2_eps_0.01_s_100/test.in
n_1000_c_100000000_g_6_f_0.3_eps_0_s_100/test.in
n_1200_c_100000000_g_14_f_0.3_eps_0.001_s_200/test.in
n_1000_c_100000000_g_6_f_0.2_eps_1e-05_s_200/test.in
n_600_c_1000000_g_6_f_0.1_eps_0.1_s_200/test.in
n_1200_c_10000000000_g_14_f_0.3_eps_0.01_s_300/test.in
n_400_c_1000000_

In [17]:
testSummary = pd.DataFrame(data = values, columns = columns)
testSummary.head()

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError
0,n_600_c_1000000_g_2_f_0.3_eps_0_s_300,527855,527843,0.128349,0.002907,0.000023
1,n_800_c_10000000000_g_14_f_0.3_eps_0.1_s_200,9999782171,9264918802,665.718123,0.004301,0.073488
2,n_800_c_10000000000_g_2_f_0.2_eps_0.1_s_200,6000016871,6000016871,0.098871,0.003442,0.000000
3,n_800_c_10000000000_g_6_f_0.1_eps_0.001_s_200,9997513189,9737508557,8.421776,0.004299,0.026007
4,n_600_c_100000000_g_14_f_0.1_eps_1e-05_s_100,100006526,99998353,6.501696,0.003328,0.000082


In [18]:
testSummary['n'] = testSummary.instanceName.apply(lambda name: re.findall(r"n_(\d+).*", name)[0])
testSummary['c'] = testSummary.instanceName.apply(lambda name: re.findall(r".*c_(\d+).*", name)[0])
testSummary.head()

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError,n,c
0,n_600_c_1000000_g_2_f_0.3_eps_0_s_300,527855,527843,0.128349,0.002907,0.000023,600,1000000
1,n_800_c_10000000000_g_14_f_0.3_eps_0.1_s_200,9999782171,9264918802,665.718123,0.004301,0.073488,800,10000000000
2,n_800_c_10000000000_g_2_f_0.2_eps_0.1_s_200,6000016871,6000016871,0.098871,0.003442,0.000000,800,10000000000
3,n_800_c_10000000000_g_6_f_0.1_eps_0.001_s_200,9997513189,9737508557,8.421776,0.004299,0.026007,800,10000000000
4,n_600_c_100000000_g_14_f_0.1_eps_1e-05_s_100,100006526,99998353,6.501696,0.003328,0.000082,600,100000000


# **Visualization**

## **Train**

### **Q-Table convergence**

In [19]:
QAbsErrors = [(q_tables[i][0], abs(q_tables[i][1] - q_tables[i-1][1])) for i in range(1, len(q_tables))]
QErrors = [(errorTable[0], errorTable[1].mean(), errorTable[1].std()) for errorTable in QAbsErrors]
QErrorsDF = pd.DataFrame(data = QErrors, columns = ['epoch', 'mean', 'std'])
QErrorsDF.head()

,epoch,mean,std
0,100,0.537744,1.267708
1,200,0.161192,0.769878
2,300,0.323505,0.805814
3,400,0.101485,0.545330
4,500,0.115152,0.541104


In [30]:
fig = px.line(QErrorsDF, x = 'epoch', y = 'mean', title = 'Q-Table Convergence')

fig.update_layout(
    yaxis_title="Q-Table Mean Absulute Error",
    xaxis_title="episode"
)

fileTemplate = "../plots/{}/QTableConvergence.{}"
fig.write_html(fileTemplate.format("html","html"), full_html=True, include_plotlyjs='cdn')
fig

## **Test**

In the cell below, we dropped all instances without a optimal solution and those instances for which we found an unfeasible solution. 

,Number of items,Capacity,count,Mean relative error (%),relativeError_std,relativeError_min,relativeError_25%,relativeError_50%,relativeError_75%,relativeError_max
0,1000,1000000,33.0,0.452904,0.013011,0.0,1.877394e-05,5.029524e-05,0.001154,0.064886
1,1000,100000000,48.0,0.541940,0.012415,0.0,2.887759e-06,9.804310e-05,0.001350,0.049627
2,1000,10000000000,23.0,1.205861,0.024171,0.0,7.332517e-10,2.199949e-09,0.006133,0.094729
3,1200,1000000,38.0,0.515211,0.013476,0.0,6.245051e-06,3.370444e-05,0.000384,0.071611
4,1200,100000000,44.0,0.930147,0.021853,0.0,2.957083e-06,7.054236e-04,0.002721,0.084606


In [21]:
testStats = testSummary[(testSummary.optimumFO != -1) & (testSummary.bestFO >= 0)].drop(['instanceName', 'optimumFO', 'bestFO'], axis = 1).groupby(['n','c']).describe()
testStats

optimumRuntime                                               \
                          count        mean          std       min       25%   
n    c                                                                         
1000 1000000               33.0    0.629272     0.911742  0.058373  0.111350   
     100000000             48.0   31.805593    61.217056  0.107452  0.715903   
     10000000000           23.0  261.991257   887.235679  0.091054  0.246487   
1200 1000000               38.0    0.772424     1.129680  0.052015  0.126794   
     100000000             44.0   34.944665    86.841742  0.131280  0.756752   
     10000000000           31.0  426.090392  1068.588748  0.113484  1.204633   
400  1000000               42.0    0.282710     0.275393  0.046710  0.088763   
     100000000             47.0    9.394258    20.743351  0.071593  0.222929   
     10000000000           34.0   70.779865   132.713759  0.071851  0.116905   
600  1000000               48.0    0.347852     0.443536  0.050456  0.088677   
     100000000             35.0   21.319913    77.437725  0.064728  0.178213   
     10000000000           24.0  175.310820   366.613185  0.077154  0.363250   
800  1000000               34.0    0.395605     0.490960  0.048589  0.115435   
     100000000             37.0   39.047706    98.802143  0.076405  0.284430   
     10000000000           39.0  278.734356   745.573079  0.068917  0.198787   

                                                     runtime            ...  \
                        50%         75%          max   count      mean  ...   
n    c                                                                  ...   
1000 1000000       0.184010    0.747961     4.335682    33.0  0.004872  ...   
     100000000     7.140235   26.159501   294.078567    48.0  0.005668  ...   
     10000000000   0.593619   27.317455  4222.267330    23.0  0.004312  ...   
1200 1000000       0.275831    0.654100     3.918329    38.0  0.005892  ...   
     100000000     8.702209   17.489976   519.782642    44.0  0.005959  ...   
     10000000000  27.523952  116.723365  4795.614831    31.0  0.005781  ...   
400  1000000       0.162916    0.371926     1.267928    42.0  0.002453  ...   
     100000000     1.314687    8.606258    99.131133    47.0  0.002661  ...   
     10000000000   0.525308   47.662029   409.610610    34.0  0.002262  ...   
600  1000000       0.118356    0.441028     1.968798    48.0  0.003336  ...   
     100000000     3.150416    8.416147   449.869251    35.0  0.003176  ...   
     10000000000   1.722071   35.333457  1088.753533    24.0  0.003357  ...   
800  1000000       0.190459    0.400434     1.929668    34.0  0.004509  ...   
     100000000     3.365852   22.694043   443.066133    37.0  0.004230  ...   
     10000000000   3.217082   64.265046  3801.790159    39.0  0.003960  ...   

                                     relativeError                           \
                       75%       max         count      mean       std  min   
n    c                                                                        
1000 1000000      0.005426  0.006597          33.0  0.004529  0.013011  0.0   
     100000000    0.006371  0.008150          48.0  0.005419  0.012415  0.0   
     10000000000  0.004873  0.006144          23.0  0.012059  0.024171  0.0   
1200 1000000      0.006507  0.008147          38.0  0.005152  0.013476  0.0   
     100000000    0.006694  0.008994          44.0  0.009301  0.021853  0.0   
     10000000000  0.006698  0.009399          31.0  0.013905  0.025357  0.0   
400  1000000      0.002736  0.003706          42.0  0.008548  0.019341  0.0   
     100000000    0.002857  0.005350          47.0  0.002826  0.008212  0.0   
     10000000000  0.002573  0.004966          34.0  0.005772  0.016561  0.0   
600  1000000      0.003612  0.007374          48.0  0.010586  0.016168  0.0   
     100000000    0.003460  0.004300          35.0  0.009324  0.020959  0.0   
     10000000000  0.003688  0.004740          2

Generating plot for relative error

In [76]:
relativeErrorDF = testStats.reset_index()[['n','c','relativeError']]
relativeErrorDF.columns = ["_".join(column) for column in relativeErrorDF.columns]
relativeErrorDF['n_'] = relativeErrorDF['n_'].apply(lambda x: int(x))
relativeErrorDF['relativeError_mean'] = relativeErrorDF['relativeError_mean'].apply(lambda x: float(x)*100)
relativeErrorDF = relativeErrorDF.rename(
    columns = {
        'relativeError_mean': 'Mean relative error (%)',
        'n_': 'Number of items',
        'c_': '0/1 Knapsack Capacity',
        'relativeError_count': 'count'
    }
)
relativeErrorDF.head()

,Number of items,0/1 Knapsack Capacity,count,Mean relative error (%),relativeError_std,relativeError_min,relativeError_25%,relativeError_50%,relativeError_75%,relativeError_max
0,1000,1000000,33.0,0.452904,0.013011,0.0,1.877394e-05,5.029524e-05,0.001154,0.064886
1,1000,100000000,48.0,0.541940,0.012415,0.0,2.887759e-06,9.804310e-05,0.001350,0.049627
2,1000,10000000000,23.0,1.205861,0.024171,0.0,7.332517e-10,2.199949e-09,0.006133,0.094729
3,1200,1000000,38.0,0.515211,0.013476,0.0,6.245051e-06,3.370444e-05,0.000384,0.071611
4,1200,100000000,44.0,0.930147,0.021853,0.0,2.957083e-06,7.054236e-04,0.002721,0.084606


In [77]:
fig = px.bar(
    relativeErrorDF, 
    x = 'Number of items', 
    y = 'Mean relative error (%)', 
    color = '0/1 Knapsack Capacity', 
    barmode = 'group', 
    hover_data = ['count'], 
    title = "Mean relative error by instances group"
)

fileTemplate = "../plots/{}/relativeError.{}"
# fig.write_image(fileTemplate.format("png","png"))
fig.write_html(fileTemplate.format("html","html"), full_html=True, include_plotlyjs='cdn')
fig

Next, we focus on those instances without optimum solutions and those instances for which the best solution was unfeasible.

In [22]:
withoutOptimumOrUnfeasible = testSummary[(testSummary.optimumFO == -1) | (testSummary.bestFO < 0)]
withoutOptimumOrUnfeasible

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError,n,c
13,n_800_c_10000000000_g_14_f_0.2_eps_0.01_s_100,-1,9997182463,-1.000000,0.004800,9.997182e+09,800,10000000000
16,n_1000_c_100000000_g_6_f_0.2_eps_1e-05_s_200,96930433,-690364053740515970,5.437214,0.006735,7.122263e+09,1000,100000000
18,n_1200_c_10000000000_g_14_f_0.3_eps_0.01_s_300,-1,9980888076,-1.000000,0.006353,9.980888e+09,1200,10000000000
75,n_600_c_10000000000_g_14_f_0.3_eps_0.01_s_100,-1,9919154826,-1.000000,0.004462,9.919155e+09,600,10000000000
96,n_600_c_10000000000_g_14_f_0.1_eps_0.01_s_200,-1,9919795158,-1.000000,0.002443,9.919795e+09,600,10000000000
...,...,...,...,...,...,...,...,...
623,n_800_c_10000000000_g_14_f_0.2_eps_0.001_s_300,-1,9997036955,-1.000000,0.004822,9.997037e+09,800,10000000000
627,n_1200_c_10000000000_g_10_f_0.3_eps_0.0001_s_300,-1,9996837153,-1.000000,0.007058,9.996837e+09,1200,10000000000
635,n_1200_c_10000000000_g_10_f_0.2_eps_1e-05_s_300,-1,9995538003,-1.000000,0.006791,9.995538e+09,1200,10000000000
636,n_600_c_10000000000_g_14_f_0.2_eps_0.0001_s_200,-1,9999534272,-1.000000,0.003874,9.999534e+09,600,10000000000


Running time comparison: exact methods x hyperheuristic

In [35]:
scatterPlotDF = testSummary[['instanceName', 'optimumRuntime', 'runtime', 'n', 'c']]
scatterPlotDF = scatterPlotDF.rename(columns = {'optimumRuntime': 'Benchmark runtime (s)', 'runtime': 'RLH runtime (s)', 'c': 'capacity'})
fig = px.scatter(scatterPlotDF, x = 'RLH runtime (s)', y = 'Benchmark runtime (s)', color = 'capacity', title = 'Running Time Comparison: RLH vs Benchmark')

fileTemplate = "../plots/{}/runningTimeComparison.{}"
# fig.write_image(fileTemplate.format("png","png"))
fig.write_html(fileTemplate.format("html","html"), full_html=True, include_plotlyjs='cdn')
fig